In [7]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

from IPython.display import clear_output

import matplotlib.pyplot as plt 

%matplotlib inline

In [8]:
def mnist_data():
    compose = transforms.Compose(
                [transforms.ToTensor(),
                 transforms.Normalize((.5,.5,.5),(.5,.5,.5))
                ])
    out_dir = './dataset'
    
    return datasets.MNIST(out_dir, transform=compose, download=True)

data = mnist_data()

Processing...
Done!


In [9]:
## Create loader with data so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
num_batches = len(data_loader)

In [10]:
class DiscriminatorNet(torch.nn.Module):
    """
    Three hidden layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        
        n_features = 784
        n_out = 1
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3))
        
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3))
        
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3))
        
        self.out = nn.Sequential(
            nn.Linear(256, n_out),
            nn.Sigmoid())
        
    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

In [11]:
class GeneratorNet(torch.nn.Module):
    """
    Three hidden layer GeneratorNetwork
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        
        n_features = 100
        n_out = 784
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2))
        
        self.hidden1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2))
        
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2))
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh())
        
    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x  

In [12]:
def noise(size):
    n = Variable(torch.randn(size, 100))
    if torch.cuda.is_available(): return n.cuda()
    return n

In [13]:
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

In [14]:
discriminator = DiscriminatorNet()
generator = GeneratorNet()  
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

In [15]:
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

In [16]:
loss = nn.BCELoss()

In [17]:
def ones_target(size):
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def zeros_target(size):
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [18]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    optimizer.zero_grad()
    
    # Train on real data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, ones_target(N))
    error_real.backward()
    
    # Train on fake data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    # Update weights with gradient
    optimizer.step()
    
    # Return error and predictions
    return (error_real + error_fake), prediction_real, prediction_fake

In [19]:
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    optimizer.zero_grad()
    
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    
    # Calculate error and backpropagate
    error = loss(prediction, ones_target(N))
    error.backward()
    
    # Updateweightswith gradient
    optimizer.step()
    
    return error

In [20]:
num_test_samples = 16
test_noise = noise(num_test_samples)

In [ ]:
# Create logger instance
logger = Logger(model_name='VGAN', data_name='MNIST')

num_epochs = 200

for epoch in range(num_epochs):
    
    for n_batch, (real_batch, _) in enumerate(data_loader):
        
        N = real_batch.size(0)
        
        # Train discriminator
        real_data = Variable(images_to_vectors(real_batch))
        if torch.cuda.is_available(): real_data = real_data.cuda()
        # .detach() is used so gradient for generator is not calculated
        fake_data = generator(noise(N)).detach()
        
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer, real_data, fake_data)
        
        # Train Generator
        fake_data = generator(noise(N))
        g_error = train_generator(g_optimizer, fake_data)
        
        # Log batch error
        # logger.log(d_error, g_error, epoch, n_batch, num_batches)
        
        # Display Progress every few batches
        if (n_batch) % 100 == 0: 
            display.clear_output(True)
            test_images = vectors_to_images(generator(test_noise)).data.cpu()
            logger.log_images(
                test_images, num_test_samples, 
                epoch, n_batch, num_batches
            );
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            ) 